In [12]:
# AS Update repeated to be removed : import requests, json

### import all the dependencies 
import requests
import csv
import json
import pandas as pd
import requests, json
import re
import configparser

In [9]:
## Define all variables 
configFile = "config.cfg"
config = configparser.ConfigParser()
config.read(configFile)


['config.cfg']

In [13]:
### Main Program starts here  - Now we will read data form CSV

customers_df = pd.read_csv("../data/Maindata_AS.csv", low_memory=False)
print('Shape of exports read from CSV : {}'.format(customers_df.shape))

Shape of exports read from CSV : (527534, 35)


In [14]:
##List the names of columns 
customers_df.columns


Index(['Unnamed: 0', 'EventModDead', 'EventModError', 'EventModFrustrated',
       'EventStart', 'EventTargetSelectorTok', 'EventTargetText', 'EventType',
       'IndvId', 'PageActiveDuration', 'PageAgent', 'PageBrowser',
       'PageDevice', 'PageDuration', 'PageId', 'PageIp', 'PageLatLong',
       'PageNumErrors', 'PageNumInfos', 'PageNumWarnings',
       'PageOperatingSystem', 'PageRefererUrl', 'PageUrl', 'SessionId',
       'UserAppKey', 'UserDisplayName', 'UserEmail', 'UserId',
       'user_anonymousId_str', 'user_firstName_str', 'user_lastName_str',
       'user_serviceName_str', 'user_userHash_str', 'user_userId_str',
       'user_username_str'],
      dtype='object')

In [15]:
len(customers_df.columns)

35

In [16]:
# AS Update: Capitalize first letter in print and sync text

## Data Cleaning and filtering 
#dropping an Column not needed 
print('number of columns  before :{}'.format(len(customers_df.columns)))
customers_df.drop('Unnamed: 0', axis=1, inplace=True)
print('number of columns  After :{}'.format(len(customers_df.columns)))

number of columns  before :35
number of columns  After :34


In [17]:
## Need to Add a column as Domain 
print('Adding domain column')

customers_df['Domain'] = customers_df['UserEmail'].str.split(pat="@", expand=True)[1]
print('Shape of exports read from CSV : {}'.format(customers_df.shape))

Adding domain column
Shape of exports read from CSV : (527534, 35)


In [18]:

# drop rows that doesn't have an email
customers_df = customers_df.dropna(subset=['UserEmail'])

# remove all the rows that use internal email or not valid email
#Read the info on emails fron config file 
dropemails= config.get('Emails', 'dropEmailList')
dropemailsList = dropemails.split(",")
external_customers_df = customers_df[~ customers_df.Domain.isin(dropemailsList)]

print('Shape of dataframe after dropping internal users : {}'.format(external_customers_df.shape))



Shape of dataframe after dropping internal users : (66210, 35)


In [19]:
#Quick preview of data
external_customers_df.head()


,EventModDead,EventModError,EventModFrustrated,EventStart,EventTargetSelectorTok,EventTargetText,EventType,IndvId,PageActiveDuration,PageAgent,...,UserEmail,UserId,user_anonymousId_str,user_firstName_str,user_lastName_str,user_serviceName_str,user_userHash_str,user_userId_str,user_username_str,Domain
3230,NaN,NaN,NaN,2018-09-08T07:29:44.963Z,NaN,NaN,navigate,5747761553080320,77531,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,adeniyi.ogundare@spcsng.com,5747761553080320,NaN,Adeniyi,ogundare,COST_INSIGHT,NaN,NaN,adeniyi.ogundare@spcsng.com,spcsng.com
3231,NaN,NaN,NaN,2018-09-08T07:29:45.115Z,NaN,NaN,navigate,5747761553080320,77531,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,adeniyi.ogundare@spcsng.com,5747761553080320,NaN,Adeniyi,ogundare,COST_INSIGHT,NaN,NaN,adeniyi.ogundare@spcsng.com,spcsng.com
3232,NaN,NaN,NaN,2018-09-08T07:29:45.275Z,NaN,NaN,load,5747761553080320,77531,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,adeniyi.ogundare@spcsng.com,5747761553080320,NaN,Adeniyi,ogundare,COST_INSIGHT,NaN,NaN,adeniyi.ogundare@spcsng.com,spcsng.com
3233,NaN,NaN,NaN,2018-09-08T07:29:55.960Z,html body #intercom%2Dcontainer%2Dbody div #in...,NaN,click,5747761553080320,77531,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,adeniyi.ogundare@spcsng.com,5747761553080320,NaN,Adeniyi,ogundare,COST_INSIGHT,NaN,NaN,adeniyi.ogundare@spcsng.com,spcsng.com
3234,NaN,NaN,NaN,2018-09-08T07:30:12.987Z,html body #intercom%2Dcontainer%2Dbody div #in...,New conversation,click,5747761553080320,77531,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,adeniyi.ogundare@spcsng.com,5747761553080320,NaN,Adeniyi,ogundare,COST_INSIGHT,NaN,NaN,adeniyi.ogundare@spcsng.com,spcsng.com


In [20]:
## Dropping unwanted columns 
external_customers_df = external_customers_df[['Domain','UserAppKey','user_serviceName_str','UserId','IndvId','SessionId',
                                       'PageUrl','PageRefererUrl','PageBrowser','PageDevice','PageDuration','PageActiveDuration',
                                        'PageLatLong',
                                       'EventStart','EventType']]
print('Shape of dataframe after dropping extra columns: {}'.format(external_customers_df.shape))                                    

Shape of dataframe after dropping extra columns: (66210, 15)


In [21]:
##Renaming columns
external_customers_df.rename(columns={'UserAppKey':'UserEmail','user_serviceName_str':'serviceName'},inplace = True)
external_customers_df.columns

Index(['Domain', 'UserEmail', 'serviceName', 'UserId', 'IndvId', 'SessionId',
       'PageUrl', 'PageRefererUrl', 'PageBrowser', 'PageDevice',
       'PageDuration', 'PageActiveDuration', 'PageLatLong', 'EventStart',
       'EventType'],
      dtype='object')

In [22]:
## Printing list of unique services we have collected data for 
list(external_customers_df.serviceName.unique())

['COST_INSIGHT', 'LOG_INTELLIGENCE', 'SDK', 'CloudAssembly', 'DISCOVERY']

In [29]:
## Dropping rows with extra services we dont care they were part of some POC 
##servciesName = list(customers_data.user_serviceName_str.unique())
print("Size of data before dropping servcies {}".format(external_customers_df.shape))
service_picked= ['LOG_INTELLIGENCE','CloudAssembly','SDK']
external_customers_df = external_customers_df[external_customers_df.serviceName.isin(service_picked)]
print("Size of data after dropping servcies {} ".format(external_customers_df.shape))

Size of data before dropping servcies (53906, 15)
Size of data after dropping servcies (9399, 15) 


In [24]:
# AS Update: Need to be moved from here and included in comp trace removal notebook

## REPLACING SERVICES NAMES to make them more meaning full
external_customers_df.replace("LOG_INTELLIGENCE","Data Logging",inplace =True)
external_customers_df.replace('CloudAssembly',"Data Package",inplace =True)
print("Size of data {}".format(external_customers_df.shape))
list(external_customers_df.serviceName.unique())

Size of data (53906, 15)


['Data Logging', 'SDK', 'Data Package']

In [25]:
# AS Update: Need to be moved from here and included in comp trace removal notebook

#Masking Company Names 
domainList  = list(external_customers_df.Domain.unique())
counter = 1
for domain in domainList  :
    newDomain = "Company" + str(counter)+".com"
    print(newDomain)
    external_customers_df.replace(domain,newDomain,inplace =True,regex=True)
    counter=counter +1  

print("Masking done")
external_customers_df.head(20)

Company1.com
Company2.com
Company3.com
Company4.com
Company5.com
Company6.com
Company7.com
Company8.com
Company9.com
Company10.com
Company11.com
Company12.com
Company13.com
Company14.com
Company15.com
Company16.com
Company17.com
Company18.com
Company19.com
Company20.com
Company21.com
Company22.com
Company23.com
Company24.com
Company25.com
Company26.com
Company27.com
Company28.com
Company29.com
Company30.com
Company31.com
Company32.com
Company33.com
Company34.com
Company35.com
Company36.com
Company37.com
Company38.com
Company39.com
Company40.com
Company41.com
Company42.com
Company43.com
Company44.com
Company45.com
Company46.com
Company47.com
Company48.com
Company49.com
Company50.com
Company51.com
Company52.com
Company53.com
Company54.com
Company55.com
Company56.com
Company57.com
Company58.com
Company59.com
Company60.com
Company61.com
Company62.com
Company63.com
Company64.com
Company65.com
Company66.com
Company67.com
Company68.com
Masking done


,Domain,UserEmail,serviceName,UserId,IndvId,SessionId,PageUrl,PageRefererUrl,PageBrowser,PageDevice,PageDuration,PageActiveDuration,PageLatLong,EventStart,EventType
6512,Company1.com,chad.tew@Company1.com,Data Logging,5650932471693312,5650932471693312,5629499534213120,https://www.mgmt.cloud.vmware.com/li/,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,20030,3459,"33.3335,-86.9591",2018-09-08T23:10:55.257Z,navigate
6513,Company1.com,chad.tew@Company1.com,Data Logging,5650932471693312,5650932471693312,5629499534213120,https://www.mgmt.cloud.vmware.com/li/,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,20030,3459,"33.3335,-86.9591",2018-09-08T23:10:56.231Z,load
6514,Company1.com,chad.tew@Company1.com,Data Logging,5650932471693312,5650932471693312,5629499534213120,https://www.mgmt.cloud.vmware.com/li/,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,20030,3459,"33.3335,-86.9591",2018-09-08T23:11:15.045Z,click
24455,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.mgmt.cloud.vmware.com/catalog/#/li...,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:03.950Z,navigate
24456,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.mgmt.cloud.vmware.com/catalog/#/li...,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:04.026Z,load
24457,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.mgmt.cloud.vmware.com/catalog/#/li...,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:08.845Z,click
24458,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.mgmt.cloud.vmware.com/catalog/#/de...,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:08.849Z,navigate
24461,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.mgmt.cloud.vmware.com/catalog/#/de...,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:17.486Z,click
24462,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.mgmt.cloud.vmware.com/catalog/#/de...,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:18.689Z,click
24463,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.mgmt.cloud.vmware.com/catalog/#/de...,https://console.cloud.vmware.com/csp/gateway/d...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:20.219Z,click


In [27]:
# AS Update: Need to be moved from here and included in comp trace removal notebook

#Masking Some Sensitive URL
external_customers_df.replace("/www.mgmt.cloud.vmware.com/","/www.cloud.dataguru.com/",inplace =True,regex=True)
external_customers_df.replace("vmware","dataguru",inplace =True,regex=True)
external_customers_df.replace("csp","bin",inplace =True,regex=True)
external_customers_df.replace("deployment","dataType",inplace =True,regex=True)
external_customers_df.replace("automation-ui","dataType-UI",inplace =True,regex=True)
external_customers_df.replace("provisioning","data-provision",inplace =True,regex=True)


In [28]:
# AS Update: print("External Customer Data Successfully Written To: external_customers.csv ")

### Writing  masked /filtered and clean data to CSV ,from here always read the local file names external_customers.csv
external_customers_df.to_csv("../data/external_customers.csv")
print("external Customer data  Successfully written to external_customers.csv ")

external Customer data  Successfully written to external_customers.csv 


In [ ]:
# AS Update: not required here, to be deleted and included in LoadExternalDataAndWork.ipynb

## Now We need to answer active users or company by unique session ID count 
##This is sample code anyway we will do the processing in another file 
df_groupby_session = external_customers_df.groupby(["Domain",'UserEmail'])['UserId'].nunique()
df_groupby_session


In [ ]:
# AS Update : not required here, to be deleted and included in LoadExternalDataAndWork.ipynb

## Now We need to answer most used service 

df_groupby_service = external_customers_df.groupby(["serviceName"])["PageUrl"].count()
df_groupby_service